# 261 Homework 1 - Big Data and Naive Bayes

HW1.0.0. Define big data. Provide an example of a big data problem in your domain of expertise. 

HW1.0.1.In 500 words (English or pseudo code or a combination) describe how to estimate the bias, the variance, the irreduciable error for a test dataset T when using polynomial regression models of degree 1, 2,3, 4,5 are considered. How would you select a model?

## Question 2: Naive Bayes

__Question 1__: Read through the provided control script (pNaiveBayes.sh) and all of its comments. When you are comfortable with their purpose and function, respond to the remaining homework questions below.

In [1]:
print "Done!"

Done!


__NB__ Certain lines appeared to be incorrect with some text based errors. The ids are listed below. I have edited the text and stored the original, unedited text as a file "enronemail_1h copy.txt"

0011.2001-06-28.SA_and_HP -> incorrect return made two lines

0001.2000-06-06.lokay -> separated text to appropriate place and made subject NA

0009.2001-06-26.SA_and_HP -> moved text to content section and made subject NA

This was manually checked using the following code:

In [4]:
with open("enronemail_1h.txt", "r") as myfile:
    for line in myfile.readlines():
        stuff = line.split('\t')
        if len(stuff) != 4:
            print stuff[0]
            print len(stuff)
            

__Question 2__: Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh will determine the number of occurrences of a single, user-specified word. Examine the word “assistance” and report your results.

   To do so, make sure that
   
   - mapper.py counts all occurrences of a single word, and
   - reducer.py collates the counts of the single word.

CROSSCHECK: >grep assistance enronemail_1h.txt|cut -d$'\t' -f4| grep assistance|wc -l
      
      8 (should really be 10 if counting occurrences)


In [10]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
findword = sys.argv[2]
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        words = re.findall(WORD_RE, line)
        occ = [i for i,x in enumerate(words) if x.lower() == findword.lower()]
        count += len(occ)
print count

Overwriting mapper.py


In [11]:
#Overwrite mapper.py
!chmod a+x mapper.py

In [12]:
%%writefile reducer.py
#!/usr/bin/python
import sys

files = sys.argv[1:]
sum = 0
for filename in files:
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            sum = sum + int(line)
print sum

Overwriting reducer.py


In [13]:
#overwrite reducer.py
!chmod a+x reducer.py

In [1]:
%%writefile pNaiveBayes.sh
## pNaiveBayes.sh
## Author: Jake Ryland Williams
## Usage: pNaiveBayes.sh m wordlist
## Input:
##       m = number of processes (maps), e.g., 4
##       wordlist = a space-separated list of words in quotes, e.g., "the and of"
##
## Instructions: Read this script and its comments closely.
##               Do your best to understand the purpose of each command,
##               and focus on how arguments are supplied to mapper.py/reducer.py,
##               as this will determine how the python scripts take input.
##               When you are comfortable with the unix code below,
##               answer the questions on the LMS for HW1 about the starter code.

## collect user input
m=$1 ## the number of parallel processes (maps) to run
wordlist=$2 ## if set to "*", then all words are used

## a test set data of 100 messages
data="enronemail_1h.txt" 

## the full set of data (33746 messages)
# data="enronemail.txt" 

## 'wc' determines the number of lines in the data
## 'perl -pe' regex strips the piped wc output to a number
linesindata=`wc -l $data | perl -pe 's/^.*?(\d+).*?$/$1/'`

## determine the lines per chunk for the desired number of processes
linesinchunk=`echo "$linesindata/$m+1" | bc`

## split the original file into chunks by line
split -l $linesinchunk $data $data.chunk.

## assign python mappers (mapper.py) to the chunks of data
## and emit their output to temporary files
for datachunk in $data.chunk.*; do
    ## feed word list to the python mapper here and redirect STDOUT to a temporary file on disk
    ####
    ####
    ./mapper.py $datachunk "$wordlist" > $datachunk.counts &
    ####
    ####
done
## wait for the mappers to finish their work
wait

## 'ls' makes a list of the temporary count files
## 'perl -pe' regex replaces line breaks with spaces
countfiles=`\ls $data.chunk.*.counts | perl -pe 's/\n/ /'`

## feed the list of countfiles to the python reducer and redirect STDOUT to disk
####
####
./reducer.py $countfiles > $data.output
####
####

## clean up the data chunks and temporary count files
#\rm $data.chunk.*
\rm enronemail_1h.txt.chunk.*

Overwriting pNaiveBayes.sh


In [2]:
!chmod a+x pNaiveBayes.sh

In [16]:
!./pNaiveBayes.sh 4 "assistance"
! echo "Number of times word appears: "
! head enronemail_1h.txt.output

Number of times word appears: 
10


## Question 3

In [1]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re


WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
findword = sys.argv[2]
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        components = line.split('\t')
        ID = components[0]
        flag = components[1]
        text = components[2] + components[3]
        word_count = {}
        for word in WORD_RE.findall(text):
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
        for word, count in word_count.iteritems():
            found_word = 0
            if word.lower() == findword.lower():
                found_word = 1
            print ID + '\t' + str(flag) + '\t' + word + '\t' + str(count) + '\t' + str(found_word)
            

Overwriting mapper.py


In [2]:
#Overwrite mapper.py
!chmod a+x mapper.py

In [13]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
from math import log

emailID = set()
spam = set()
vocab = set()
spamCount = 0
spamWordCount = 0
hamCount= 0
hamWordCount = 0
foundWord = set()

files = sys.argv[1:]
for filename in files:
    with open(filename, 'r') as myfile:
        for line in myfile.readlines():
            components = line.split('\t')
            ID = components[0]
            flag = int(components[1])
            word = components[2]
            count = int(components[3])
            findword = int(components[4])
            if findword == 1:
                foundWord.add(word)
            emailID.add(ID)
            if flag == 1:
                spam.add(ID)
                spamCount += count
                if findword == 1:
                    spamWordCount += count
            else:
                hamCount += count
                if findword == 1:
                    hamWordCount += count

            
priorSpam = float(len(spam))/len(emailID)
priorHam = float(len(emailID) - len(spam))/len(emailID)


condProbSpam = float(spamWordCount)/spamCount
condProbHam = float(hamWordCount)/hamCount

#classify new emails
#in this case I decided to read the original file, rather than reassembling the emails from the reducer step.
#This increases modularity because in practice we would report our results on a validation or test set as opposed
#to the training set. 
WORD_RE = re.compile(r"[\w']+")    
with open('enronemail_1h.txt', 'r') as myfile:
    for line in myfile.readlines():
        components = line.split('\t')
        ID = components[0]
        trueLabel = components[1]
        text = components[2] + ' ' + components[3]
        spamScore = log(priorSpam)
        hamScore = log(priorHam)
        for word in WORD_RE.findall(text):
            if word in foundWord and condProbSpam != 0:
                spamScore += log(condProbSpam)
            if word in foundWord and condProbHam != 0:
                hamScore += log(condProbHam)
        predicted = 0
        if spamScore > hamScore:
            predicted = 1
        print ID + '\t' + str(trueLabel) + '\t' + str(predicted)

Overwriting reducer.py


In [14]:
#overwrite reducer.py
!chmod a+x reducer.py

In [15]:
!./pNaiveBayes.sh 4 "assistance"

In [17]:
with open('enronemail_1h.txt.output', 'r') as myfile:
    count = 0
    for line in myfile.readlines():
        components = line.split('\t')
        if int(components[1]) != int(components[2]):
            count += 1
    print count

40


## Question 4

HW1.4. Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by a list of one or more user-specified words. Examine the words “assistance”, “valium”, and “enlargementWithATypo” and report your results
   To do so, make sure that

   - mapper.py counts all occurrences of a list of words, and
   - reducer.py 

   performs the multiple-word multinomial Naive Bayes classification via the chosen list.
No smoothing is needed in this HW.

In [15]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re


WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
findwords = re.split(" ",sys.argv[2].lower())
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        components = line.split('\t')
        ID = components[0]
        flag = components[1]
        text = components[2] + components[3]
        word_count = {}
        for word in WORD_RE.findall(text):
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
        for word, count in word_count.iteritems():
            found_word = 0
            if word.lower() in findwords:
                found_word = 1
            print ID + '\t' + str(flag) + '\t' + word + '\t' + str(count) + '\t' + str(found_word)

Overwriting mapper.py


In [16]:
#Overwrite mapper.py
!chmod a+x mapper.py

In [32]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
from math import log

emailID = set()
spam = set()
vocab = set()
spamCount = 0
spamWordCount = {}
hamCount= 0
hamWordCount = {}
foundWord = set()

files = sys.argv[1:]
for filename in files:
    with open(filename, 'r') as myfile:
        for line in myfile.readlines():
            components = line.split('\t')
            ID = components[0]
            flag = int(components[1])
            word = components[2]
            count = int(components[3])
            findword = int(components[4])
            if findword == 1:
                foundWord.add(word)
            emailID.add(ID)
            if flag == 1:
                spam.add(ID)
                spamCount += count
                if findword == 1:
                    if word not in spamWordCount:
                        spamWordCount[word] = count
                    else:
                        spamWordCount[word] += count
            else:
                hamCount += count
                if findword == 1:
                    if word not in hamWordCount:
                        hamWordCount[word] = count
                    else:
                        hamWordCount[word] += count

            
priorSpam = float(len(spam))/len(emailID)
priorHam = float(len(emailID) - len(spam))/len(emailID)


condProbSpam = {}
condProbHam = {}

for word in foundWord:
    if word in spamWordCount:
        condProbSpam[word] = float(spamWordCount[word])/spamCount
    if word in hamWordCount:
        condProbHam[word] = float(hamWordCount[word])/hamCount

        
#classify new emails
#in this case I decided to read the original file, rather than reassembling the emails from the reducer step.
#This increases modularity because in practice we would report our results on a validation or test set as opposed
#to the training set. 
WORD_RE = re.compile(r"[\w']+")    
with open('enronemail_1h.txt', 'r') as myfile:
    for line in myfile.readlines():
        components = line.split('\t')
        ID = components[0]
        trueLabel = components[1]
        text = components[2] + ' ' + components[3]
        spamScore = log(priorSpam)
        hamScore = log(priorHam)
        for word in WORD_RE.findall(text):
            if word in foundWord and word in condProbSpam:
                spamScore += log(condProbSpam[word])
            if word in foundWord and word in condProbHam:
                hamScore += log(condProbHam[word])
        predicted = 0
        if spamScore > hamScore:
            predicted = 1
        print ID + '\t' + str(trueLabel) + '\t' + str(predicted)

Overwriting reducer.py


In [33]:
#overwrite reducer.py
!chmod a+x reducer.py

In [34]:
!./pNaiveBayes.sh 4 'assistance valium enlargementWithATypo'

In [35]:
with open('enronemail_1h.txt.output', 'r') as myfile:
    count = 0
    for line in myfile.readlines():
        components = line.split('\t')
        if int(components[1]) != int(components[2]):
            count += 1
    print count

40


-0.82098055207
-0.579818495253


## Question 5

HW1.5. Provide a mapper/reducer pair that, when executed by pNaiveBayes.sh
   will classify the email messages by all words present.
   To do so, make sure that

   - mapper.py counts all occurrences of all words, and
   - reducer.py performs a word-distribution-wide Naive Bayes classification.

In [3]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re


WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        components = line.split('\t')
        ID = components[0]
        flag = components[1]
        text = components[2] + components[3]
        word_count = {}
        for word in WORD_RE.findall(text):
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
        for word, count in word_count.iteritems():
            print ID + '\t' + str(flag) + '\t' + word + '\t' + str(count)

Overwriting mapper.py


In [4]:
#change permissions on mapper
!chmod a+x mapper.py

In [51]:
%%writefile reducer.py
#!/usr/bin/python
import sys
import re
from math import log

emails = set()
spams = set()
words_in_corpus = set()
spam_counts = {}
ham_counts = {}

files = sys.argv[1:]
for filename in files:
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            components = line.split('\t')
            ID = components[0]
            flag = int(components[1])
            word = components[2]
            count = int(components[3])
            emails.add(ID)
            words_in_corpus.add(word)
            if flag == 1:
                spams.add(ID)
                if word not in spam_counts:
                    spam_counts[word] = count
                else:
                    spam_counts[word] += count
            else:
                if word not in ham_counts:
                    ham_counts[word] = count
                else:
                    ham_counts[word] += count
            
priorSpam = float(len(spams))/len(emails)
priorHam = float(len(emails) - len(spams))/len(emails)

condProbSpam = {}
condProbHam = {}

for word in words_in_corpus:
    if word in spam_counts:
        spam_tokens = spam_counts[word] + 1
    else:
        spam_tokens = 1
    condProbSpam[word] = float(spam_tokens)/(sum(spam_counts.values()) + len(words_in_corpus))
    if word in ham_counts:
        ham_tokens = ham_counts[word] + 1
    else:
        ham_tokens = 1
    condProbHam[word] = float(ham_tokens)/(sum(ham_counts.values()) + len(words_in_corpus))

#classify new emails
#in this case I decided to read the original file, rather than reassembling the emails from the reducer step.
#This increases modularity because in practice we would report our results on a validation or test set as opposed
#to the training set. 
WORD_RE = re.compile(r"[\w']+")    
with open('enronemail_1h.txt', 'r') as myfile:
    for line in myfile.readlines():
        components = line.split('\t')
        ID = components[0]
        trueLabel = components[1]
        text = components[2] + ' ' + components[3]
        spamScore = log(priorSpam)
        hamScore = log(priorHam)
        for word in WORD_RE.findall(text):
            if word in condProbSpam:
                spamScore += log(condProbSpam[word])
            if word in condProbHam:
                hamScore += log(condProbHam[word])
        predicted = 0
        if spamScore > hamScore:
            predicted = 1
        print ID + '\t' + str(trueLabel) + '\t' + str(predicted)

Overwriting reducer.py


In [52]:
#change permissions on reducer
!chmod a+x reducer.py

In [53]:
!./pNaiveBayes.sh 4

In [57]:
with open('enronemail_1h.txt.output', 'r') as myfile:
    count = 0
    for line in myfile.readlines():
        components = line.split('\t')
        true = int(components[1])
        pred = int(components[2])
        count += (true - pred)
    print count

0


In [ ]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re


WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[1]
with open (filename, "r") as myfile:
    for line in myfile.readlines():
        components = line.split('\t')
        ID = components[0]
        flag = components[1]
        text = components[2] + components[3]
        word_count = {}
        for word in WORD_RE.findall(text):
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
        for word, count in word_count.iteritems():
            print ID + '\t' + str(flag) + '\t' + word + '\t' + str(count)